In [17]:
import os
import sys
sys.path.append(os.getcwd())
sys.path.append(os.getcwd() + '/../src')


In [18]:
import logging
import wandb
from concurrent.futures import ThreadPoolExecutor
import multiprocessing


In [19]:
# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout),
        logging.FileHandler('download.log')
    ]
)
logger = logging.getLogger(__name__)

# Set multiprocessing start method to 'spawn'
multiprocessing.set_start_method("spawn", force=True)

In [20]:
# Configuration
download_dir = "/scratch/downloaded_artifacts"
entity = "wlp9800-new-york-university"
project_name = "oho_exps"
group_name = "fixed_search-2_fd29c503918d4cc7ace8d3ee2a21778e"
max_download_workers = 20

In [21]:
# Ensure download directory exists
os.makedirs(download_dir, exist_ok=True)

In [22]:
# Function to download a single run's artifact
def download_artifact(run_data):
    run_id = run_data["id"]
    config = run_data["config"]
    try:
        api = wandb.Api()
        artifact = api.artifact(f'{entity}/{project_name}/logs_{run_id}:v0')
        artifact_dir = os.path.join(download_dir, artifact.name)
        artifact.download(root=artifact_dir)
        logger.info(f"Downloaded {artifact.name} to {artifact_dir}")
        return {
            "run_id": run_id,
            "artifact_dir": artifact_dir,
            "config": config,
            "status": "success"
        }
    except Exception as e:
        logger.error(f"Error downloading artifact for run {run_id}: {str(e)}")
        return {
            "run_id": run_id,
            "artifact_dir": None,
            "config": config,
            "status": f"error: {str(e)}"
        }

In [23]:
api = wandb.Api()
runs = api.runs(
    path=f"{entity}/{project_name}",
    filters={"group": group_name}
)

# Prepare run data
run_data = [{"id": run.id, "config": run.config} for run in runs]
logger.info(f"Found {len(run_data)} runs to download")


2025-04-21 03:43:07,597 - INFO - Found 2000 runs to download


In [24]:
with ThreadPoolExecutor(max_workers=max_download_workers) as executor:
    download_results = list(executor.map(download_artifact, run_data))

wandb:   1 of 1 files downloaded.  


2025-04-21 03:43:10,352 - INFO - Downloaded logs_7rvtffzk:v0 to /scratch/downloaded_artifacts/logs_7rvtffzk:v0


wandb:   1 of 1 files downloaded.  


2025-04-21 03:43:10,370 - INFO - Downloaded logs_12leuth0:v0 to /scratch/downloaded_artifacts/logs_12leuth0:v0


wandb:   1 of 1 files downloaded.  


...

2025-04-21 03:50:03,467 - INFO - Downloaded logs_m80phxh8:v0 to /scratch/downloaded_artifacts/logs_m80phxh8:v0


wandb:   1 of 1 files downloaded.  


2025-04-21 03:50:03,485 - INFO - Downloaded logs_p8apgtw3:v0 to /scratch/downloaded_artifacts/logs_p8apgtw3:v0


wandb:   1 of 1 files downloaded.  


2025-04-21 03:50:03,639 - INFO - Downloaded logs_hvmcqo0c:v0 to /scratch/downloaded_artifacts/logs_hvmcqo0c:v0


In [25]:
# Save download results for the processing script, including group name
import pickle
results_file = os.path.join(download_dir, f'download_results_{group_name}.pkl')
with open(results_file, 'wb') as f:
    pickle.dump(download_results, f)
logger.info(f"Saved download results to {results_file}")

2025-04-21 03:50:03,723 - INFO - Saved download results to /scratch/downloaded_artifacts/download_results_fixed_search-2_fd29c503918d4cc7ace8d3ee2a21778e.pkl
